In [1]:
#Kako OCR api
import json

import requests
import sys

import cv2

In [2]:
LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()


    return requests.post(API_URL, headers=headers, files={"image": data})

In [6]:
# def main():
#     if len(sys.argv) != 3:
#         print("Please run with args: $ python example.py /path/to/image appkey")
#     image_path, appkey = '/content/drive/MyDrive/필요한이미지.jpg', '자기가 받은 Rest API 키'

image_path, appkey = 'data-files/identifyCard.png', '3606a90ae3f0a7d4df69fab8ccaaa457' # 손병우 REST API
resize_impath = kakao_ocr_resize(image_path)
if resize_impath is not None:
    image_path = resize_impath
    # print("원본 대신 리사이즈된 이미지를 사용합니다.")

output = kakao_ocr(image_path, appkey).json()
#아스키코드 해제해줘야 한국어로 나옴
print("[OCR] output:\n{}\n".format(json.dumps(output, ensure_ascii=False,sort_keys=True, indent=2)))

[OCR] output:
{
  "result": [
    {
      "boxes": [
        [
          206,
          117
        ],
        [
          224,
          117
        ],
        [
          224,
          134
        ],
        [
          206,
          134
        ]
      ],
      "recognition_words": [
        "12"
      ]
    },
    {
      "boxes": [
        [
          239,
          116
        ],
        [
          265,
          117
        ],
        [
          264,
          133
        ],
        [
          238,
          132
        ]
      ],
      "recognition_words": [
        "94"
      ]
    },
    {
      "boxes": [
        [
          277,
          112
        ],
        [
          397,
          116
        ],
        [
          396,
          139
        ],
        [
          277,
          136
        ]
      ],
      "recognition_words": [
        "627721-12"
      ]
    }
  ]
}



In [11]:
type(output)

dict

In [12]:
output.keys()

dict_keys(['result'])

In [13]:
type(output["result"])

list

In [14]:
len(output['result'])

42

In [4]:
output['result'][0]

{'boxes': [[72, 13], [118, 13], [118, 31], [72, 31]],
 'recognition_words': ['모바일']}

In [7]:
for item in output['result']:
    print(item['recognition_words'])
    

['12']
['94']
['627721-12']
